In [6]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

In [7]:
df = pd.read_csv("Titanic-Dataset.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


To start, I will need to determine which feature to use.

PassengerID, Name, Ticket can be dropped first since they have very small influence on surviving. Also dropped  Cabin since it has many null values. 

In [9]:
df = df.drop(columns=["PassengerId", "Ticket", "Name", "Cabin"])


I will want to use Sex, Pclass, Age, Fare, and Embarked.

In [10]:
# performing one-hot encoding
df = pd.get_dummies(df, columns=["Sex", "Embarked"], drop_first=True)


In [11]:
#handle missing age
df["Age"] = df["Age"].fillna(df["Age"].median())
df["Fare"] = df["Fare"].fillna(df["Fare"].median())

In [12]:
df.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,1,0,1
1,1,1,38.0,1,0,71.2833,0,0,0
2,1,3,26.0,0,0,7.9250,0,0,1
3,1,1,35.0,1,0,53.1000,0,0,1
4,0,3,35.0,0,0,8.0500,1,0,1


In [13]:
features = [
    "Pclass",
    "Age",
    "Fare",
    "Sex_male",
    "Embarked_Q",
    "Embarked_S"
]

X = df[features]
y = df["Survived"]


In [14]:
model = LogisticRegression(max_iter=1000)
model.fit(X, y)

pd.Series(model.coef_[0], index=X.columns).sort_values()

Sex_male     -2.488470
Pclass       -1.131345
Embarked_S   -0.489260
Age          -0.032808
Embarked_Q   -0.002370
Fare          0.000039
dtype: float64

In [15]:
model_2 = LogisticRegression(penalty="l1", solver="liblinear")
model_2.fit(X, y)

pd.Series(model_2.coef_[0], index=X.columns)

Pclass       -1.080268
Age          -0.030284
Fare          0.000546
Sex_male     -2.503170
Embarked_Q    0.000000
Embarked_S   -0.434839
dtype: float64

In [16]:
#Dropped Embarked_Q  due to l1
features = ["Sex_male", "Pclass", "Age", "Fare", "Embarked_S"]
X = df[features]
y = df["Survived"]

In [17]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [18]:

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(
        penalty="l1",
        solver="liblinear",
        C=1.0
    ))
])

In [19]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 LogisticRegression(penalty='l1', solver='liblinear'))])

In [20]:
y_pred = pipe.predict(X_val)
accuracy_score(y_val, y_pred)

0.770949720670391

In [21]:

confusion_matrix(y_val, y_pred)

array([[92, 18],
       [23, 46]])

In [22]:
y_prob = pipe.predict_proba(X_val)[:, 1]
roc_auc_score(y_val, y_prob)

0.8366271409749672

In [23]:
coefs = pipe.named_steps["model"].coef_[0]
pd.Series(coefs, index=features).sort_values()

Sex_male     -1.228314
Pclass       -0.958146
Age          -0.437712
Embarked_S   -0.248928
Fare          0.000000
dtype: float64

In [24]:
print(X.columns)

Index(['Sex_male', 'Pclass', 'Age', 'Fare', 'Embarked_S'], dtype='object')
